go through real dataset and mark artefacts, recompute fixed_name_count with ignored artefacts

In [3]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm

if os.environ["HOME"] == "/home/vcvrcek":
    csv_path = Path("/home/vcvrcek/Python/spectra_DL/labels/")
    data_folder = Path("/diska/vcvrcek/data/")
    real_memmap_filename = data_folder.joinpath("real_data/harps-nonan-stable.dat")
elif os.environ["HOME"] == "/home/cv":
    csv_path = Path("/media/cv/big_daddy/ESO/data/metadata/")
    data_folder = Path("/media/cv/big_daddy/ESO/data/")
    real_memmap_filename = data_folder.joinpath("real_data/harps-nonan-stable.dat")

csv_real_file = csv_path.joinpath("harps_labels_complete.csv")

if os.environ["HOME"] == "/home/vcvrcek":
    project_folder = Path("/home/vcvrcek/Python/spectra_DL/")
elif os.environ["HOME"] == "/home/cv":
    project_folder = Path("/home/cv/Dropbox/PHD/Python/ESO/spectra_DL/")


print(project_folder.joinpath("datasets/"))
sys.path.append(str(project_folder.joinpath("datasets/")))
print(sys.path)
from Dataset_real_memmap import harps_real_memmap

/home/cv/Dropbox/PHD/Python/ESO/spectra_DL/datasets
['/home/cv/Dropbox/PHD/Python/ESO/spectra_DL/tools/catalog_tools', '/home/cv/anaconda3/envs/spectra_DL_env_2/lib/python39.zip', '/home/cv/anaconda3/envs/spectra_DL_env_2/lib/python3.9', '/home/cv/anaconda3/envs/spectra_DL_env_2/lib/python3.9/lib-dynload', '', '/home/cv/.local/lib/python3.9/site-packages', '/home/cv/anaconda3/envs/spectra_DL_env_2/lib/python3.9/site-packages', '/home/cv/Dropbox/PHD/Python/ESO/spectra_DL/datasets', '/home/cv/Dropbox/PHD/Python/ESO/spectra_DL/datasets']


In [2]:
dataset = harps_real_memmap(
    memmap_filename=real_memmap_filename,
    dataset_name="real",
    csv_file=csv_real_file,
    median_norm=False,
)

is_artefact = np.zeros(len(dataset), dtype=bool)
for i in tqdm(range(len(dataset))):
    spec = dataset[i][0].numpy()

    med = np.median(spec * harps_real_memmap.get_artifact_mask(1))
    mean = np.mean(spec)

    if med < 50 or mean < 0:
        is_artefact[i] = True

Dataloader: total spectra in file:267361 going to use 267361
--------------> None


100%|██████████| 267361/267361 [22:48<00:00, 195.30it/s]


In [16]:
mask = ~df['is_artefact']
counts = df.loc[mask, 'target_name_fixed'].value_counts()
df.loc[mask, 'counts_wout_artefact'] = counts.reindex(df.loc[mask, 'target_name_fixed']).fillna(0).values
# df.loc[mask, 'counts_wout_artefact'] = counts.reindex(df.loc[mask, 'target_name_fixed']).values


In [15]:
mask = ~df['is_artefact']
counts = df.loc[mask, 'target_name_fixed'].value_counts()
print(counts)
print(df.loc[mask, 'target_name_fixed'])
print(df.shape)

HD128621        19675
HD10700          9199
HD20794          6766
HD190248         5704
PROCYON          5700
                ...  
HIP20187            1
HD156098F6IV        1
N210014             1
GJ4309B             1
HIP21332            1
Name: target_name_fixed, Length: 6966, dtype: int64
0         HD203608
1         HD114613
2          HIP5158
3          HD17051
4          HD17051
            ...   
267356     LTT3864
267357      HR4468
267358      HR4963
267359      HR5035
267360     C153012
Name: target_name_fixed, Length: 264040, dtype: object
(267361, 26)


In [17]:
# output the new dataframe
csv_out = csv_path.joinpath("harps_artefacts_marked.csv")
df.to_csv(csv_out, index=False)